In [4]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from src import config
from src.utils import *
from tensorflow.keras.models import load_model

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time
import colorsys  

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def get_rainbow_color(hue_offset, brightness=1.0):
    r, g, b = colorsys.hsv_to_rgb(hue_offset, 1.0, brightness)
    return (int(b * 255), int(g * 255), int(r * 255))

def get_hand_coords(landmarks, image_shape):
    h, w, c = image_shape
    coords = []
    for lm in landmarks.landmark:
        coords.append((int(lm.x * w), int(lm.y * h)))
    return coords

def draw_artistic_trail(image, history_list, trace_lifetime, current_time, base_hue_offset):
    
    for i, snapshot in enumerate(history_list):
        coords_curr, timestamp = snapshot
        
        age = current_time - timestamp
        life_ratio = 1.0 - (age / trace_lifetime) 
        life_ratio = max(0, min(1, life_ratio))
        
        if life_ratio <= 0: continue

        hue = (base_hue_offset + (i * 0.02)) % 1.0
        color = get_rainbow_color(hue, brightness=life_ratio)
        
        thickness = int(4 * life_ratio) + 1 
        
        radius = int(3 * life_ratio)

        for connection in mp_holistic.HAND_CONNECTIONS:
            pt1 = coords_curr[connection[0]]
            pt2 = coords_curr[connection[1]]
            cv2.line(image, pt1, pt2, color, thickness)
           
            if radius > 0:
                cv2.circle(image, pt1, radius, color, -1)
                cv2.circle(image, pt2, radius, color, -1)
        
        if i > 0:
            coords_prev = history_list[i-1][0]
            for j in range(21):
                pt_a = coords_prev[j]
                pt_b = coords_curr[j]
                cv2.line(image, pt_a, pt_b, color, thickness)

sequence = []
sentence = []
threshold = 0.4

history_right = [] 
history_left = [] 

trace_lifetime = 1.5
sampling_rate = 2 
frame_counter = 0
global_hue = 0.0

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        image, results = media_pipe_detection(frame, holistic)
        
        current_time = time.time()
        frame_counter += 1
        global_hue = (global_hue + 0.01) % 1.0
        
        if results.right_hand_landmarks:
            if frame_counter % sampling_rate == 0:
                coords = get_hand_coords(results.right_hand_landmarks, image.shape)
                history_right.append((coords, current_time))
        
        if results.left_hand_landmarks:
            if frame_counter % sampling_rate == 0:
                coords = get_hand_coords(results.left_hand_landmarks, image.shape)
                history_left.append((coords, current_time))

        history_right = [snap for snap in history_right if (current_time - snap[1]) < trace_lifetime]
        history_left = [snap for snap in history_left if (current_time - snap[1]) < trace_lifetime]

        draw_artistic_trail(image, history_right, trace_lifetime, current_time, global_hue)
        draw_artistic_trail(image, history_left, trace_lifetime, current_time, global_hue + 0.5)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1766499698.602162 2163400 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1
W0000 00:00:1766499698.726754 2163856 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766499698.749429 2163863 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766499698.751994 2163857 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766499698.752089 2163858 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766499698.752129 2163861 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support 